# Iyzico Datathon - Time Series Modelling

In [82]:
# Base packages
import numpy as np
import pandas as pd
import itertools
import warnings
from tqdm import tqdm

# Data Visualization
from matplotlib import pyplot as plt
import seaborn as sns

# Modelling
import pmdarima as pm
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Evaluation
from sklearn.metrics import mean_absolute_error

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", lambda x: '%.1f' % x)
warnings.filterwarnings('ignore')

## Data Manipulation

In [4]:
data = pd.read_csv("C:/Users/MONSTER/Desktop/train.csv")

def get_season(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else: # 10, 11, 12
        return 'Q4'

data['month_id'] = data['month_id'].astype(str)
data['year'] = data['month_id'].str[:4].astype(int)
data['month'] = data['month_id'].str[4:].astype(int)
data['season'] = data['month'].apply(get_season)
data['month_id'] = pd.to_datetime(data['month_id'], format='%Y%m')


column_order = ['month_id', 'merchant_id', 'year', 'month', 'season', 'merchant_source_name', 'settlement_period', 'working_type', 'mcc_id', 'merchant_segment', 'net_payment_count']
data = data[column_order]
data = data.sort_values(by='month_id').reset_index(drop=True)

# Segment column
data.loc[data['merchant_segment'].isin(["Segment - 1", "Segment - 2", "Segment - 3"]), 'merchant_segment'] = "otherSegments123"
data.loc[data['merchant_segment'] == "Segment - 4", 'merchant_segment'] = "StandartSegment4"

# negative values in target 
data.loc[data['net_payment_count'] < 0, 'net_payment_count'] = 0

# mcc_id to integer
data['mcc_id'] = data['mcc_id'].str.replace('mcc_', '').astype(int)

# Working type
data.loc[data['working_type'].isin(["Working Type - 1", "Working Type - 3", "Working Type - 4"]), 'working_type'] = "otherWorkingTypes134"

# settlement_period
data.loc[data['settlement_period'].isin(["Settlement Period - 2", "Settlement Period - 3"]), 'settlement_period'] = "otherSettlementPeriods23"

# Time series data
time_series = data.groupby(["month_id", "merchant_id"]).agg({"net_payment_count": "sum"}).reset_index()

In [5]:
data.head()

,month_id,merchant_id,year,month,season,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,2020-01-01,merchant_66740,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 6,130,StandartSegment4,6
1,2020-01-01,merchant_12444,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 2,153,StandartSegment4,3
2,2020-01-01,merchant_40154,2020,1,Q1,Merchant Source - 2,Settlement Period - 1,Working Type - 5,168,StandartSegment4,3
3,2020-01-01,merchant_33179,2020,1,Q1,Merchant Source - 3,otherSettlementPeriods23,Working Type - 2,25,otherSegments123,787
4,2020-01-01,merchant_16977,2020,1,Q1,Merchant Source - 2,Settlement Period - 1,Working Type - 6,31,StandartSegment4,5


In [6]:
time_series.head()

,month_id,merchant_id,net_payment_count
0,2020-01-01,merchant_10005,3
1,2020-01-01,merchant_10034,5
2,2020-01-01,merchant_10052,5
3,2020-01-01,merchant_10078,134
4,2020-01-01,merchant_10134,3


### 1.1 Time Series Data for Modelling 

In [7]:
# Create a date range for all months
all_months = pd.date_range(start='2020-01-01', end='2023-09-01', freq='MS')

# Create a DataFrame with all combinations of customer_id and all_months
all_customers = time_series['merchant_id'].unique()
all_combinations = pd.MultiIndex.from_product([all_customers, all_months], names=['merchant_id', 'month_id'])
temp_df = pd.DataFrame(index=all_combinations).reset_index()

# Merge the complete_df with the original df
ts_df = temp_df.merge(time_series, on=['merchant_id', 'month_id'], how='left')

# Fill missing sales with zero
ts_df['net_payment_count'] = ts_df['net_payment_count'].fillna(0)


In [11]:
ts_df.head()

,merchant_id,month_id,net_payment_count
0,merchant_10005,2020-01-01,3.0
1,merchant_10005,2020-02-01,4.0
2,merchant_10005,2020-03-01,3.0
3,merchant_10005,2020-04-01,0.0
4,merchant_10005,2020-05-01,0.0


In [12]:
last_three_months = ts_df['month_id'].max() - pd.DateOffset(months=3)
new_customer_df = ts_df[ts_df['month_id'] < last_three_months].groupby('merchant_id').filter(lambda x: x['net_payment_count'].sum() == 0)
new_customer_ids = new_customer_df['merchant_id'].unique()

## 2. Modelling

### 2.1 Holt-Winters' Modelling

In [49]:
def tes_optimizer(train, test):
    model_holt_winters = ExponentialSmoothing(train, trend='add', 
                                              seasonal_periods=12).fit(optimized=True)
    
    best_alpha = model_holt_winters.params['smoothing_level']
    best_beta = model_holt_winters.params['smoothing_trend']
    best_gamma = model_holt_winters.params['smoothing_seasonal']

    forecast = model_holt_winters.forecast(len(test))

    mae = mean_absolute_error(test, forecast)

    if np.isnan(model_holt_winters.params['smoothing_seasonal']):
        best_gamma = 0
    
    # print(f"parameters: a = {best_alpha}, b = {best_beta}, g = {best_gamma}, mae = {mae}")

    return best_alpha, best_beta, best_gamma, mae


In [50]:
forecast_df = pd.DataFrame(columns=['month_id', 'merchant_id', 'forecast'])

iteration_control = 0
total_mae = 0

for customer_id in tqdm(ts_df['merchant_id'].unique(), desc="Processing Customers"):
    try:
        customer_data = ts_df[ts_df['merchant_id'] == customer_id]
        customer_data = customer_data.set_index('month_id')
        customer_data = customer_data.drop('merchant_id', axis=1)

        # Ensure the sales column is numeric and the index is datetime
        customer_data['net_payment_count'] = pd.to_numeric(customer_data['net_payment_count'], errors='coerce')
        customer_data.index = pd.to_datetime(customer_data.index)

        if customer_id in new_customer_ids:
            moving_average_october = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"] + customer_data['net_payment_count']["2023-07-01"] + customer_data['net_payment_count']["2023-06-01"]) / 4
            moving_average_november = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"] + customer_data['net_payment_count']["2023-07-01"]) / 3 
            moving_average_december = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"]) / 2
            
            moving_averages_df = pd.DataFrame({
                'month_id': ["2023-10-01", "2023-11-01", "2023-12-01"],
                'merchant_id': [customer_id] * 3,
                'forecast': [moving_average_october, moving_average_november, moving_average_december]
            })
            
            forecast_df = forecast_df.append(moving_averages_df, ignore_index=True)
            
        else:
            train = customer_data[:"2023-05-01"]
            test = customer_data["2023-06-01":]

            y_pred_len = len(test)
            best_alpha, best_beta, best_gamma, best_mae = tes_optimizer(train, test)

            tes_model = ExponentialSmoothing(customer_data, trend="add", seasonal="add", seasonal_periods=12).\
                fit(smoothing_level=best_alpha, smoothing_slope=best_beta, smoothing_seasonal=best_gamma)
            
            y_pred = tes_model.forecast(3)
            y_pred = pd.DataFrame(y_pred, columns=['forecast'])

            # print(f"Customer {customer_id} - MAE: {best_mae}")

            y_pred = np.array(y_pred).flatten()

            forecast_dates = pd.date_range(start=customer_data.index[-1], periods=len(y_pred) + 1, freq='MS')[1:]
            forecast_df_temp = pd.DataFrame({
                'month_id': forecast_dates,
                'merchant_id': [customer_id] * len(y_pred), 
                'forecast': y_pred
            })

            forecast_df = forecast_df.append(forecast_df_temp, ignore_index=True)

            iteration_control += 1
            total_mae += best_mae

    except Exception as e:
        print(f"Error processing in customer: {customer_id}: {e}")
        break

# After all forecasts have been appended to forecast_df
avg_mae = total_mae / iteration_control
forecast_df['forecast'] = forecast_df['forecast'].apply(lambda x: 0 if x < 1 else x)
print(f"Average MAE: {avg_mae}")

Processing Customers: 100%|██████████| 26060/26060 [31:10<00:00, 13.93it/s]

Average MAE: 50.485981803654035


In [51]:
forecast_df_new = forecast_df.copy()

In [54]:
forecast_df_new.head()

,month_id,merchant_id,forecast
0,2023-10-01 00:00:00,merchant_10005,0.0
1,2023-11-01 00:00:00,merchant_10005,0.0
2,2023-12-01 00:00:00,merchant_10005,0.0
3,2023-10-01 00:00:00,merchant_10034,0.0
4,2023-11-01 00:00:00,merchant_10034,0.0


In [55]:
forecast_df_new["month_id"] = pd.to_datetime(forecast_df_new["month_id"], format='%Y-%m-%d')

In [56]:
forecast_df_new['date_id'] = forecast_df_new['month_id'].apply(lambda x: x.strftime('%Y%m') if x.month in [10, 11, 12] else '')
forecast_df_new['general_merchant_id'] = forecast_df_new['date_id'] + forecast_df_new['merchant_id']

In [90]:
forecast_df_new.to_csv("C:/Users/MONSTER/Desktop/forecast.csv", index=False)

### 2.2 ARIMA Modelling

In [85]:
forecast_df_arima = pd.DataFrame(columns=['month_id', 'merchant_id', 'forecast'])

iteration_control = 0
total_mae = 0

for customer_id in tqdm(ts_df['merchant_id'].unique(), desc="Processing Customers"):
    try:
        customer_data = ts_df[ts_df['merchant_id'] == customer_id]
        customer_data = customer_data.set_index('month_id')
        customer_data = customer_data.drop('merchant_id', axis=1)

        # Ensure the sales column is numeric and the index is datetime
        customer_data['net_payment_count'] = pd.to_numeric(customer_data['net_payment_count'], errors='coerce')
        customer_data.index = pd.to_datetime(customer_data.index)

        if customer_id in new_customer_ids:
            moving_average_october = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"] + customer_data['net_payment_count']["2023-07-01"] + customer_data['net_payment_count']["2023-06-01"]) / 4
            moving_average_november = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"] + customer_data['net_payment_count']["2023-07-01"]) / 3 
            moving_average_december = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"]) / 2
            
            moving_averages_df = pd.DataFrame({
                'month_id': ["2023-10-01", "2023-11-01", "2023-12-01"],
                'merchant_id': [customer_id] * 3,
                'forecast': [moving_average_october, moving_average_november, moving_average_december]
            })
            
            forecast_df_arima = forecast_df_arima.append(moving_averages_df, ignore_index=True)
            
        else:
            arima_model = ARIMA(customer_data, order=(1, 1, 1)).fit()
            
            y_pred = arima_model.get_forecast(3).predicted_mean
            y_pred = pd.DataFrame(y_pred)

            # print(f"Customer {customer_id} - MAE: {best_mae}")

            y_pred = np.array(y_pred).flatten()

            forecast_dates = pd.date_range(start=customer_data.index[-1], periods=len(y_pred) + 1, freq='MS')[1:]

            forecast_df_temp = pd.DataFrame({
                'month_id': forecast_dates,
                'merchant_id': [customer_id] * len(y_pred), 
                'forecast': y_pred
            })

            forecast_df_arima = forecast_df_arima.append(forecast_df_temp, ignore_index=True)

            iteration_control += 1
            total_mae += best_mae

            # if iteration_control == 100:
            #     print(forecast_df_arima.head(200))
            #     break

    except Exception as e:
        print(f"Error processing in customer: {customer_id}: {e}")
        break

# After all forecasts have been appended to forecast_df
avg_mae = total_mae / iteration_control
forecast_df_arima['forecast'] = forecast_df_arima['forecast'].apply(lambda x: 0 if x < 1 else x)
print(f"Average MAE: {avg_mae}")

Processing Customers: 100%|██████████| 26060/26060 [39:02<00:00, 11.12it/s] 

Average MAE: 7.383240546304094


In [88]:
forecast_df_arima.head()

,month_id,merchant_id,forecast
0,2023-10-01 00:00:00,merchant_10005,0.0
1,2023-11-01 00:00:00,merchant_10005,0.0
2,2023-12-01 00:00:00,merchant_10005,0.0
3,2023-10-01 00:00:00,merchant_10034,0.0
4,2023-11-01 00:00:00,merchant_10034,0.0


In [89]:
forecast_df_arima_new = forecast_df_arima.copy()

In [92]:
forecast_df_arima_new['date_id'] = forecast_df_arima_new['month_id'].apply(lambda x: x.strftime('%Y%m') if x.month in [10, 11, 12] else '')
forecast_df_arima_new['general_merchant_id'] = forecast_df_arima_new['date_id'] + forecast_df_arima_new['merchant_id']

AttributeError: 'str' object has no attribute 'month'

In [91]:
forecast_df_arima_new.to_csv("C:/Users/MONSTER/Desktop/forecast_arima.csv", index=False)